In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
filename = "final_df.json"
df = pd.read_json(path+filename, lines=True)

In [3]:
#Divido il dataframe per intervalli settimanali, riduco le colonne a quelle che mi initeressano
df['time_utc'] = pd.to_datetime(df['utc_datetime_str'])
df_small = df[['time_utc','GPT_label','polarity','text']].copy()
week_intervals = pd.date_range(start=df_small['time_utc'].min(), end=df_small['time_utc'].max(), freq='7D')

week_df_list = []
for i in range(len(week_intervals) - 1):
    start_date = week_intervals[i]
    end_date = week_intervals[i + 1]
    week_data = df_small[(df_small['time_utc'] >= start_date) & (df_small['time_utc'] < end_date)].copy()
    week_df_list.append(week_data)
week_df_list.append(df_small[(df_small['time_utc'] >= end_date) & (df_small['time_utc'] <= df_small['time_utc'].max())].copy())

In [4]:
topics = list(df_small['GPT_label'].unique())

In [5]:
#ottengo un dizionario in cui le chiavi sono i topic e i valori sono liste di DFs. Ogni lista contiene i DFs per quei topic divisi per settimana
dfs_per_weeks_and_topics = {}
for topic in topics:
    dfs_per_weeks_and_topics[topic] = []
    for weekly_df in week_df_list:
        dfs_per_weeks_and_topics[topic] .append(weekly_df[weekly_df['GPT_label'] == topic])
    

In [12]:
Nwords_per_week_t = {}
for topic in topics:
    Nwords_per_week_t[topic] = []
    for topic_weekly_df in dfs_per_weeks_and_topics[topic]:
        total_len = 0
        for text in list(topic_weekly_df['text']):
            total_len += len(text.split(" "))
        Nwords_per_week_t[topic].append(total_len)

In [13]:
def plot1(weekly_nwords_list, topic):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=week_intervals,
        y=weekly_nwords_list,
        name=f'n words per week {topic}',
        orientation='v',
        text=week_intervals,
        textposition='outside', 
        marker=dict(
            color='rgba(246, 78, 139, 0.6)',
            line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        )
    ))
    fig.update_layout(
        title=dict(
            text=f'<b>N words per week \n {topic}',  # Title text
            x=0.5,  # x=0.5 centers the title
            xanchor='center',  # 'center' ensures the title is centered
            font=dict(
                size=12,
                color='black',
                family="Arial, sans-serif",
            )
            )
    )
    fig.update_layout(
        width=1000,   # Larghezza in pixel
        height=600   # Altezza in pixel
    )
    fig.show()

In [14]:
for topic in topics:
    plot1(Nwords_per_week_t[topic], topic)